### Homework 5: Question search engine

Remeber week01 where you used GloVe embeddings to find related questions? That was.. cute, but far from state of the art. It's time to really solve this task using context-aware embeddings.

__Warning:__ this task assumes you have seen `seminar.ipynb`!

In [ ]:
%pip install --upgrade transformers datasets accelerate deepspeed transformers[sentencepiece]
import torch
import torch.nn as nn
import torch.nn.functional as F
import transformers
import datasets
import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.4 MB/s 
     |████████████████████████████████| 451 kB 65.3 MB/s 
     |████████████████████████████████| 191 kB 61.5 MB/s 
     |████████████████████████████████| 709 kB 54.1 MB/s 
     |████████████████████████████████| 7.6 MB 53.9 MB/s 
     |████████████████████████████████| 182 kB 81.1 MB/s 
     |████████████████████████████████| 132 kB 83.1 MB/s 
     |████████████████████████████████| 212 kB 81.0 MB/s 
     |████████████████████████████████| 127 kB 81.4 MB/s 
     |████████████████████████████████| 54 kB 3.2 MB/s 
     |████████████████████████████████| 145 kB 76.8 MB/s 
     |████████████████████████████████| 1.3 MB 68.9 MB/s 
  Created wheel for deepspeed: filename=deepspeed-0.7.6-py3-none-any.whl size=714425 sha256=832e2c3c4e48c68ec8ba044c7014a2900df33362f469d755b6331f03bd67407b
  Stored in directory: /root/.cache/pip/wheels/a8/8

### Load data and model

In [ ]:
qqp = datasets.load_dataset('SetFit/qqp')
print('\n')
print("Sample[0]:", qqp['train'][0])
print("Sample[3]:", qqp['train'][3])

Extracting data files #1:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #0:   0%|          | 0/1 [00:00<?, ?obj/s]

Extracting data files #2:   0%|          | 0/1 [00:00<?, ?obj/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/SetFit___json/SetFit--qqp-e0faace7f63208a7/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]



Sample[0]: {'text1': 'How is the life of a math student? Could you describe your own experiences?', 'text2': 'Which level of prepration is enough for the exam jlpt5?', 'label': 0, 'idx': 0, 'label_text': 'not duplicate'}
Sample[3]: {'text1': 'What can one do after MBBS?', 'text2': 'What do i do after my MBBS ?', 'label': 1, 'idx': 3, 'label_text': 'duplicate'}


In [ ]:
model_name = "gchhablani/bert-base-cased-finetuned-qqp"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

Downloading:   0%|          | 0.00/320 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/890 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/433M [00:00<?, ?B/s]

### Tokenize the data

In [ ]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)

  0%|          | 0/364 [00:00<?, ?ba/s]

  0%|          | 0/391 [00:00<?, ?ba/s]

  0%|          | 0/41 [00:00<?, ?ba/s]

In [ ]:
print(repr(qqp_preprocessed['train'][0]['input_ids'])[:100], "...")

[101, 1731, 1110, 1103, 1297, 1104, 170, 12523, 2377, 136, 7426, 1128, 5594, 1240, 1319, 5758, 136,  ...


### Task 1: evaluation (1 points)

We randomly chose a model trained on QQP - but is it any good?

One way to measure this is with validation accuracy - which is what you will implement next.

Here's the interface to help you do that:

In [ ]:
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=1, shuffle=False, collate_fn=transformers.default_data_collator
)

In [ ]:
for batch in val_loader:
     break  # here be your training code
print("Sample batch:", batch)

with torch.no_grad():
  predicted = model(
      input_ids=batch['input_ids'],
      attention_mask=batch['attention_mask'],
      token_type_ids=batch['token_type_ids']
  )

print('\nPrediction (probs):', torch.softmax(predicted.logits, dim=1).data.numpy())

Sample batch: {'labels': tensor([0]), 'idx': tensor([0]), 'input_ids': tensor([[  101,  2009,  1132,  2170,   118,  4038,  1177,  2712,   136,   102,
          2009,  1132,  1117, 10224,  4724,  1177,  2712,   136,   102,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,   

__Your task__ is to measure the validation accuracy of your model.
Doing so naively may take several hours. Please make sure you use the following optimizations:

- run the model on GPU with no_grad
- using batch size larger than 1
- use optimize data loader with num_workers > 1
- (optional) use [mixed precision](https://pytorch.org/docs/stable/notes/amp_examples.html)


In [ ]:
def calculate_accuracy(loader, model):
    model.eval()
    model.to('cuda')

    preds = []
    labels = []

    i = 0
    for batch in tqdm.tqdm(loader):
        i += 1
        with torch.inference_mode():
            pred = model(
                input_ids=batch['input_ids'].to('cuda'),
                attention_mask=batch['attention_mask'].to('cuda'),
                token_type_ids=batch['token_type_ids'].to('cuda'),
            )

            labels.append(batch['labels'])
            preds.append(torch.softmax(pred.logits, dim=1).detach().cpu())
        # if i == 3:
        #    break


    preds = torch.concat(preds, dim=0).argmax(dim=1)
    labels = torch.concat(labels, dim=0)

    accuracy = (preds == labels).float().mean()
    return accuracy


val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=12, shuffle=False, 
    collate_fn=transformers.default_data_collator, num_workers=2
)

accuracy = calculate_accuracy(val_loader, model)


100%|██████████| 3370/3370 [05:34<00:00, 10.07it/s]


In [ ]:
assert 0.9 < accuracy < 0.91

### Task 2: train the model (5 points)

For this task, you have two options:

__Option A:__ fine-tune your own model. You are free to choose any model __except for the original BERT.__ We recommend [DeBERTa-v3](https://huggingface.co/microsoft/deberta-v3-base). Better yet, choose the best model based on public benchmarks (e.g. [GLUE](https://gluebenchmark.com/)).

You can write the training code manually or use transformers.Trainer (see [this example](https://github.com/huggingface/transformers/blob/main/examples/pytorch/text-classification)). Please make sure that your model's accuracy is at least __comparable__ with the above example for BERT.


__Option B:__ compare at least 3 pre-finetuned models (in addition to the above BERT model). For each model, report (1) its accuracy, (2) its speed, measured in samples per second in your hardware setup and (3) its size in megabytes. Please take care to compare models in equal setting, e.g. same CPU / GPU. Compile your results into a table and write a short (~half-page on top of a table) report, summarizing your findings.

In [ ]:
from transformers import Trainer, TrainingArguments

In [ ]:
model_name = "microsoft/deberta-v3-base"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

Downloading:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/579 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.8/dist-packages/transformers/convert_slow_tokenizer.py:446: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Downloading:   0%|          | 0.00/371M [00:00<?, ?B/s]

Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForSequenceClassification: ['lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.bias', 'mask_predictions.dense.weight', 'mask_predictions.dense.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.bias', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'mask_predictions.LayerNorm.weight', 'lm_predictions.lm_head.dense.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a

In [ ]:
qqp_preprocessed = qqp.map(preprocess_function, batched=True)
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=32, shuffle=False, collate_fn=transformers.default_data_collator
)


  0%|          | 0/364 [00:00<?, ?ba/s]

  0%|          | 0/391 [00:00<?, ?ba/s]

  0%|          | 0/41 [00:00<?, ?ba/s]

In [ ]:
args = TrainingArguments(
    output_dir='/output',
    overwrite_output_dir=True,
    do_train=True,
    per_device_train_batch_size=48,
    dataloader_num_workers=2,
    max_steps=500,
    learning_rate=1e-5
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=qqp_preprocessed['train'],
    tokenizer=tokenizer,
    data_collator=transformers.default_data_collator,
)

max_steps is given, it will override any value given in num_train_epochs


In [ ]:
result = trainer.train()
trainer.save_model()

The following columns in the training set don't have a corresponding argument in `DebertaV2ForSequenceClassification.forward` and have been ignored: text1, text2, label_text, idx. If text1, text2, label_text, idx are not expected by `DebertaV2ForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 363846
  Num Epochs = 1
  Instantaneous batch size per device = 48
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 1
  Total optimization steps = 500
  Number of trainable parameters = 184423682


Step,Training Loss
500,0.399100


Saving model checkpoint to /output/checkpoint-500
Configuration saved in /output/checkpoint-500/config.json
Model weights saved in /output/checkpoint-500/pytorch_model.bin
tokenizer config file saved in /output/checkpoint-500/tokenizer_config.json
Special tokens file saved in /output/checkpoint-500/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Saving model checkpoint to /output
Configuration saved in /output/config.json
Model weights saved in /output/pytorch_model.bin
tokenizer config file saved in /output/tokenizer_config.json
Special tokens file saved in /output/special_tokens_map.json


In [ ]:
MAX_LENGTH = 128
def preprocess_function(examples):
    result = tokenizer(
        examples['text1'], examples['text2'],
        padding='max_length', max_length=MAX_LENGTH, truncation=True
    )
    result['label'] = examples['label']
    return result

qqp_preprocessed = qqp.map(preprocess_function, batched=True)
val_set = qqp_preprocessed['validation']
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=32, shuffle=False, collate_fn=transformers.default_data_collator
)


  0%|          | 0/364 [00:00<?, ?ba/s]

In [ ]:
print('Val accuracy (finetuned DeBERTa-v3)', calculate_accuracy(val_loader, model))

100%|██████████| 1264/1264 [06:12<00:00,  3.40it/s]

Val accuracy (finetuned DeBERTa-v3) tensor(0.8617)


### Task 3: try the full pipeline (2 points)

Finally, it is time to use your model to find duplicate questions.
Please implement a function that takes a question and finds top-5 potential duplicates in the training set. For now, it is fine if your function is slow, as long as it yields correct results.

Showcase how your function works with at least 5 examples.

In [ ]:
qqp['train'][:20]

{'text1': ['How is the life of a math student? Could you describe your own experiences?',
  'How do I control my horny emotions?',
  'What causes stool color to change to yellow?',
  'What can one do after MBBS?',
  'Where can I find a power outlet for my laptop at Melbourne Airport?',
  "How not to feel guilty since I am Muslim and I'm conscious we won't have sex together?",
  'How is air traffic controlled?',
  'What is the best self help book you have read? Why? How did it change your life?',
  "Can I enter University of Melbourne if I couldn't achieve the guaranteed marks in Trinity College Foundation?",
  'Do you need a passport to go to Jamaica from the United States?',
  'What is the district of Edgware and how does the lifestyle compare to the London Borough of Islington?',
  "What will be Hillary Clinton's policy towards India if she becomes president?",
  'What is the responsibility of SAP ERP key user?',
  'Which is the best book to study TENSOR for general relativity from b

In [ ]:
def qqp_pipeline(quest: str):
    results = []

    for i in range(len(qqp['train'])):

        for text_type in ['text1', 'text2']:
            sample = qqp['train'][i][text_type]
            
            tokenized = tokenizer(quest, sample, padding='max_length', max_length=MAX_LENGTH, truncation=True)
            batch = transformers.default_data_collator([tokenized])

            with torch.inference_mode():
                pred = model(
                    input_ids=batch['input_ids'].to('cuda'),
                    attention_mask=batch['attention_mask'].to('cuda'),
                    token_type_ids=batch['token_type_ids'].to('cuda'),
                )
                pred_label = torch.softmax(pred.logits, dim=1).argmax(dim=1)
                if pred_label == 1:
                    results.append(sample)
            
            if len(results) == 5:
                print('Source question: ', quest)
                print('Found duplicate questions:')
                print(*[res_sample + '\n' for res_sample in results])

                return 
                
    print('Source question: ', quest)
    print('Found duplicate questions:')
    print(*[res_sample + '\n' for res_sample in results])


model.eval()
model.to('cuda')
qqp_pipeline("How do I control my emotions?")
qqp_pipeline("When the WW2 ends?")
qqp_pipeline("What country is the best for life?")
qqp_pipeline("What do you think about Tesla?")
qqp_pipeline("Is there a life after death?")

Source question:  How do I control my emotions?
Found duplicate questions:
How do I control my emotions and anger?
 How one can control impulsive emotions?
 How can I control my emotions?
 How do I change mood instantly?
 How do i control emotions at work place or public place?

Source question:  When the WW2 ends?
Found duplicate questions:
What overall caused WWII to end?
 How close we are to see 3rd world war?
 How close we are to see 3rd world war?
 How close we are to see 3rd world war?
 How close we are to see 3rd world war?

Source question:  What country is the best for life?
Found duplicate questions:
What are some of the best countries in the work and stay?
 Which is the best country for live in the world peacefully?
 Which is the best country to live in the world?
 Which is the most beautiful country in the world in terms of natural beauty?
 Which are the best countries to visit?



__Bonus:__ for bonus points, try to find a way to run the function faster than just passing over all questions in a loop. For isntance, you can form a short-list of potential candidates using a cheaper method, and then run your tranformer on that short list. If you opted for this solution, please keep both the original implementation and the optimized one - and explain briefly what is the difference there.